In [1]:
import string
import pandas as pd
import numpy as np
import nltk
import ssl

from bs4 import BeautifulSoup

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, hamming_loss
from sklearn.metrics import hamming_loss
from sklearn.model_selection import train_test_split

import catboost as ctb
from catboost.utils import eval_metric
from gensim.models import Word2Vec

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
import torch
import torch.nn as nn

import lightning as L
    
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gorinenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gorinenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Постановка задачи

В второй части w2v_base_line.ipynb мы составили базовый pipeline обучение модели CatBoostClassifier на базе признаков, извлеченных с использованием Word2Vec. В этом документе попробуем использовать рекурентные сети для предсказания меток-тегов и сравним скорость обучения модели и ее предсказательную способность. 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Предобработка текста

In [3]:
file_path = 'data/stackoverflow_posts.csv'
# file_path = 'data/stackexchange_posts.csv'

nrows = 100_000
raw_df = pd.read_csv(file_path)
df = raw_df[~raw_df['Tags'].isna()]

df = df.iloc[:nrows, :]
df.reset_index(inplace=True)
# Всего - 912090

print(f'samples count {df.shape[0]}')

samples count 100000


In [4]:
df.fillna('', inplace=True)

In [5]:
def parse_tags(value):
    tags = value.replace('<', '').split('>')
    return [tag for tag in tags if tag]

    
df["Tags"] = df["Tags"].apply(lambda x: parse_tags(x))

In [6]:
stop_words = set(stopwords.words('russian'))
punctuation = set(string.punctuation)


class TextPreProcessor:
    def __init__(self, tokenizer, stemmer=None, morph=None):
        self.tokenizer = tokenizer
        self.stemmer = stemmer
        self.morph = morph


    def tokenize(self, text: str):      
        text = text.lower()
      
        doc = BeautifulSoup(text, 'lxml')
        text = doc.text
        
        tokens = self.tokenizer.tokenize(text)
        
        words = [word for word in tokens if word not in stop_words and word not in punctuation]
        
        if self.morph:
            words = [self.morph.parse(word)[0].normal_form for word in words]

        if self.stemmer:
            words = [self.stemmer.stem(word) for word in words]

        return words
    
class NltkTokenizer:    
    def tokenize(self, text: str):      
        return list(word_tokenize(text))

Построим словарь, который сопоставляет словам некие индексы. Мы используем специальный токен **unk**, который будет возвращен, если слово отсутствует в словаре. Внутри себя функция подсчитывает частоту появления каждого токена, потом слова сортируются по убыванию частоты и из уже отсортированного словаря строиться vocab. Таким образом самые часто встречаемые слова будут в начале(специальный токен **unk** имеет индекс 0). В словаре также имеется токен **pad**, который дополняет предложение до определенной длины, если оно меньше, например, если мы условились, что длина последовательности равна 10, то предложение 'here is the an example' будет закодировано так 'here is the an example pad pad pad pad pad'. Только на месте чисел должны стоять их индексы в словаре.


In [7]:
from torchtext.vocab import build_vocab_from_iterator


def preprocessor(text):
    if not isinstance(text, str):
          return text
      
    tokenizer = TextPreProcessor(tokenizer=NltkTokenizer())    
    words = tokenizer.tokenize(text)
    return words
    
def yield_tokens(data_iter):
    for tokens in data_iter:
        yield preprocessor(tokens)
        
df['Body_cleaned'] = df['Body'].apply(lambda x: preprocessor(x))
        
vocab = build_vocab_from_iterator(yield_tokens(df['Body_cleaned']), specials=['<unk>', '<pad>'])
vocab.set_default_index(vocab['<unk>'])

/var/folders/g5/8xrz2mg176j1yxttghxsl4gc0000gn/T/ipykernel_29557/2654832454.py:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  doc = BeautifulSoup(text, 'lxml')


In [8]:
print(f'Размер словаря {len(vocab)}')
print(f'Индексы слов {vocab(["javascript", "принтер", "abra-cadabra", "<unk>"])}')
print(f'Слово с индексом 100 - "{vocab.lookup_token(100)}"')

PAD_INDEX=vocab['<pad>']
UNK_INDEX=vocab['<unk>']

Размер словаря 316302
Индексы слов [252, 10873, 0, 0]
Слово с индексом 100 - "value="


In [9]:
multi_label = MultiLabelBinarizer()
Y = multi_label.fit_transform(list(df["Tags"]))

Наконец, создадим WordDataset и DataLoader с использованием функции collate_batch, которая будет формировать пакеты наших данных. Обратите внимание, что архитектура RNN требует, чтобы все предложения в пакете данных для обучения на каком-то шаге имели одинаковую длину. Поэтому в процессе формирования пакетов преобразуем токены текста в индексы подготовленного словаря, а затем дополним последовательности индексом специального символа **pad**.

In [10]:

class WordDataset:
    def __init__(self, data,  encode_labels=None):
        self.data = np.array(data['Body_cleaned'])
        self.encode_labels = None if encode_labels is None else np.array(encode_labels)
        assert len(self.data) == len(self.encode_labels)

    def __getitem__(self, idx: int):
        if self.encode_labels is None:
            return None, self.data[idx]
        
        return self.encode_labels[idx], self.data[idx]

    def __len__(self) -> int:
        return len(self.data)

In [11]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

random_state=42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

text_pipeline = lambda x: vocab(x)

def collate_batch(batch):
    label_list, text_list, offsets = None, [], []
    
    # Формируем списки тензоров
    for _label, _text in batch:
        # Режим тренировки
        if _label is not None:
            if label_list is None:
                label_list = []
            label_list.append(torch.FloatTensor(_label))
        
        processed_text = torch.tensor(vocab(_text), dtype=torch.int64)
        text_list.append(processed_text)
        
        offsets.append(processed_text.size(0))
    
    # Преобразуем списки тензоров в тензор и выравниваем последовательности
    if label_list:
        label_list = torch.FloatTensor(np.array(label_list)).to(device)
        
    offsets = torch.tensor(np.array(offsets), dtype=torch.int64).to(device)
    text_list = pad_sequence(text_list, padding_value=PAD_INDEX).permute(1, 0).to(device)
    
    # Сортируем
    offsets, ordering = torch.sort(offsets, dim=0, descending=True)
    text_list = text_list[ordering]
    if label_list is not None:
        label_list = label_list[ordering].to(device)
   
        
    return text_list.to(device), label_list


X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.1, random_state=random_state, shuffle=False)
# , num_workers=11, persistent_workers=True
train_loader = DataLoader(WordDataset(X_train, y_train), batch_size=8, shuffle=True, drop_last=True, collate_fn=collate_batch, num_workers=11, persistent_workers=True)
valid_loader = DataLoader(WordDataset(X_test, y_test), batch_size=8, shuffle=False, drop_last=True, collate_fn=collate_batch, num_workers=11, persistent_workers=True)

# Обучение модели

## Определение модели

Для предсказывания меток попробуем использовать рекурентные нейронные сети.

In [12]:
EMBEDDING_DIM = 200
RNN_HIDDEN_DIM = 100
RNN_NUM_LAYERS = 2

class TextClassificationModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embedding = nn.Embedding(num_embeddings=len(vocab), embedding_dim=EMBEDDING_DIM, padding_idx = PAD_INDEX)
        self.rnn = nn.RNN(input_size=EMBEDDING_DIM, hidden_size=RNN_HIDDEN_DIM, num_layers=RNN_NUM_LAYERS, batch_first=True)
        self.linear = nn.Linear(RNN_HIDDEN_DIM, len(multi_label.classes_))
        
    def forward(self, X_batch):
        embeddings = self.embedding(X_batch)
        output, _ = self.rnn(embeddings)
        
        return self.linear(output[:,-1])

In [13]:
import torch.nn.functional as F
import lightning as L


class TextClassificationLightningModule(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.save_hyperparameters()
        
        self.model = model
        self.sigmoid = torch.nn.Sigmoid()
        
    def predict_step(self, batch, batch_idx):
        X, y = batch
        logits = self.model(X)
        
        prob = self.sigmoid(logits)        
        preds = torch.round(prob)
        
        return prob, preds

    def training_step(self, batch, *args):
        X, y = batch
        logits = self.model(X)
        
        loss = F.cross_entropy(logits, y)        
        return loss
    
    def test_step(self, batch, batch_idx):
        X, y = batch
        logits = self.model(X)
        
        prob = self.sigmoid(logits)        
        preds = torch.round(prob)
        
        test_loss = F.mse_loss(preds, y)
        self.log("test_loss", test_loss)
        
    # def validation_step(self, batch, batch_idx):
    #     X, y = batch
    #     logits = self.model(X)
        
    #     prob = self.sigmoid(logits)        
    #     preds = torch.round(prob)
        
    #     val_loss = F.mse_loss(preds, y)
    #     self.log("val_loss", val_loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [14]:
# train model
model = TextClassificationLightningModule(TextClassificationModel())
trainer = L.Trainer(max_epochs=1, default_root_dir="data/models/", profiler="simple")
# trainer.fit(model, train_loader, valid_loader)
trainer.fit(model, train_loader)

/Users/gorinenko/src/multi_tagging_classification/.venv/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/gorinenko/src/multi_tagging_classification/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoa

In [ ]:
trainer.test(model, valid_loader)

/Users/gorinenko/src/multi_tagging_classification/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.0273505300283432
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0273505300283432}]

## Оценка модели

Оценим метрики scikit learn

## Предсказания модели

In [ ]:
valid_df = df[df['Tags'].isna()]

In [ ]:
import random

valid_df = raw_df[~raw_df['Tags'].isna()]
valid_df.reset_index(inplace=True)


row_num = random.randint(0, valid_df['Body'].shape[0])
text = valid_df['Body'][row_num]
tokens = preprocessor(text)


print(f'Текст: \n{text}\n')

processed_text = torch.tensor(vocab(tokens), dtype=torch.int64)
processed_text = pad_sequence([processed_text], padding_value=PAD_INDEX).permute(1, 0).to(device)
model.model.eval()
with torch.no_grad():
    logits = model.model(processed_text)
    
    prob = torch.nn.Sigmoid()(logits)        
    preds = torch.round(prob)


labels = multi_label.inverse_transform(preds.reshape(1, -1))
print(f'Предсказанные теги: \n{labels}\n')

len(labels[0])

Текст: 
<p>Всем привет!) У меня установлен ocStore 2.3.0.2.3.
Я хочу, чтобы, когда клиент авторизовался, было написан его ник, а не "личный кабинет". Очень долго уже ищу, не могу найти. Нашел вот такую строку, но куда ее вставлять не знаю:
$data['text_username'] = $this->customer->getFirstName();</p>

<p>Спасибо за любую помощь!)</p>


Предсказанные теги: 
[('.net', 'ajax', 'android', 'c', 'c#', 'c++', 'css', 'delphi', 'gwt', 'html', 'html5', 'ide', 'java', 'javascript', 'joomla', 'jquery', 'linux', 'mysql', 'pascal', 'php', 'python', 'sql', 'sql-server', 'ubuntu', 'visual-basic', 'visual-studio', 'winapi', 'windows', 'winforms', 'wpf', 'алгоритм', 'ассемблер', 'база-данных', 'книги', 'массивы', 'регулярные-выражения')]



36

In [ ]:
len(labels[0])


36